In [1]:
#Importação das bibliotecas necessárias para rodar o código
%matplotlib inline

import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import re
import Augmentor

In [2]:
#Define o mapa de cores para geração da máscara
def colorir(cat):
    h, w = cat.shape[:2]
    msk = np.zeros((h,w,3), dtype = 'uint8')
    msk[cat == 1] = [0,255,0]
    return(msk)

In [6]:
#Definição das pastas necessárias para rodar o script
img_dir = '../../Datasets/SoybeanTS/GP/' 
s_dir = '../../Datasets/SoybeanTS/augmented/'

anotacao = '../data/via_anot_GP.csv'

In [7]:
#Apresenta as imagens dentro do diretório com o respectivo formato da imagem
img_list = os.listdir(img_dir)
img_list = [v for v in img_list if v.endswith('.JPG')]
img_list = sorted(img_list)
print(len(img_list))

28


In [5]:
csv_file = pd.read_csv(anotacao,index_col = False) #Faz a leitura do arquivo .csv

images = []
for img in img_list:
    lista = csv_file[csv_file.filename == img] #Recebe os dados encontrados dentro do csv para a imagem em questão
    tamanho = lista.shape

    if tamanho[0] > 1:
        imagem = cv2.imread(os.path.join(img_dir, img)) #Carrega a imagem em questão

        # Importa a mascara do indice de vegetacao:
        msk = np.zeros(imagem.shape[:2], dtype = 'float32') #Carrega as informações do tamanho da imagem
        msk_name = os.path.join(img_dir, img.replace('.JPG', '_msk.png')) #Faz a junção do diretório com o nome da imagem, alterando seu formato


        if tamanho[0] > 1:
            for i in range(tamanho[0]):
                line = lista.iloc[i,:] #Recebe todas as marcações realizadas dentro daquela imagem
                region_shape = line.region_shape_attributes #Informa a posição onde o ponto se encontra
                region_attributes = (line.region_attributes) #Informa a classe do ponto
                region_attributes = re.findall('"([^"]*)"', region_attributes) #Faz a separação para encontrar a qual classe pertence o ponto marcado
                coords = [float(s) for s in re.findall(r'-?\d+\.?\d*', region_shape)] #Encontrando valores de x e y
                cx = int(coords[0]) #Coordenadas no eixo X
                cy = int(coords[1]) #Coordenadas no eixo y

                cv2.circle(msk, (cx,cy), 10, 1, -1)

        print(msk_name) #Apresenta o nome da máscara com o diretório a ser salvo
        cv2.imwrite(msk_name, msk.astype('uint8')) #Realiza o salvamento do background
        
        im_l = [imagem, msk]
        images.append(im_l)


../../Datasets/SoybeanTS/GP/DJI_0344_msk.png
../../Datasets/SoybeanTS/GP/DJI_0345_msk.png
../../Datasets/SoybeanTS/GP/DJI_0346_msk.png
../../Datasets/SoybeanTS/GP/DJI_0347_msk.png
../../Datasets/SoybeanTS/GP/DJI_0348_msk.png


In [8]:
p = Augmentor.DataPipeline(images)

p.crop_by_size(1, 512, 512, centre = False)
p.flip_left_right(probability=0.5)
p.flip_top_bottom(probability=0.5)
p.random_distortion(probability=0.1, grid_width=4, grid_height=4, magnitude=4)
p.resize(probability=1.0, width=512, height=512)

In [9]:
augmented_images = p.sample(100)

In [10]:
try:
    os.mkdir(s_dir)
except:
    pass
for i, img in enumerate(augmented_images):
    img_name = 'RGB_{0:04d}_img.jpg'.format(i + 1)
    msk_name = 'RGB_{0:04d}_msk.png'.format(i + 1)
    col_name = 'RGB_{0:04d}_col.png'.format(i + 1)
    cv2.imwrite(os.path.join(s_dir, img_name), img[0])
    cv2.imwrite(os.path.join(s_dir, msk_name), img[1])    
    cv2.imwrite(os.path.join(s_dir, col_name), colorir(img[1]))
